In [55]:
import markovify
import pandas as pd
import os
import re
import pylab
import numpy as np
import numpy.random
import matplotlib.pyplot as plt
from PIL import Image

def find_section(word_list, link):
    section=""
    file_names=os.listdir(link)
    for i in file_names:
        f= open(link+i, encoding="utf8")
        text=f.read()
        s=re.search(re.escape(word_list[0])+r"(.*)"+re.escape(word_list[1]),text.replace("\n"," "))
        if not s is None:
            section=section+s.group(1)
        f.close()
    return section

def make_article(reference_link, section_str_dict,section_len_dict):
    whole_article=""
    for i in section_str_dict.keys():
        section_input=""
        for j in section_str_dict.get(i):
            section_input=section_input+find_section(j,reference_link)
        model=markovify.Text(section_input)
        section_output=""
        for t in range(section_len_dict.get(i)):
            section_output=section_output+" "+model.make_sentence()
        if i=="RESULTS":
            if " table" in section_output.lower():
                section_output=section_output+"\n"+pd.DataFrame(np.random.random((5,5))).to_latex()
            if " figure" in section_output.lower():
                x = np.random.randn(5000)
                y = np.random.randn(5000)
                heatmap, xedges, yedges = np.histogram2d(x, y, bins=50)
                plt.clf()
                plt.imshow(heatmap.T)
                plt.savefig(os.getcwd()+"\\result_image.png")
                section_output=section_output+"\n"+"\\begin{figure}[H]\n\centering\n\includegraphics*[width=1\\textwidth]{result_image.png}\n\end{figure}"
        whole_article=whole_article+" "+"\section{"+i+"}\n"+section_output+"\n"
    return whole_article

In [ ]:
reference_link="C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\Bioinformatics\\"
section_str_dict={"Motivation":[("Motivation","Results")],"Results":[("Results","Availability")],"Availability":[("Availability","Contact")],"INTRODUCTION":[("INTRODUCTION","RESULTS")],"METHODS":[("METHODS","RESULT"),("METHODS","RESULTS"),("METHOD","RESULT"),("METHOD","RESULTS")],"RESULTS":[("RESULTS","DISCUSSION"),("RESULT","DISCUSSION"),("RESULTS","DISCUSSIONS"),("RESULT","DISCUSSIONS")],"DISCUSSION":[("DISCUSSIONS","CONCLUSION"),("DISCUSSION","CONCLUSION"),("DISCUSSIONS","CONCLUSIONS"),("DISCUSSION","CONCLUSIONS")],"CONCLUSIONS":[("CONCLUSIONS","AVAILABILITY"),("CONCLUSION","ACKNOWLEDGMENTS"),("CONCLUSIONS","FUNDING")]}
section_len_dict={"Motivation":3,"Results":3,"Availability":1,"INTRODUCTION":10,"METHODS":10,"RESULTS":15,"DISCUSSION":5,"CONCLUSIONS":5}
result=make_article(reference_link,section_str_dict,section_len_dict)
print(result)

In [56]:
intro=""
link="C:\\Users\\murat_gga8ya6\\Desktop\\RSG\\hackathon-rsg-paper\\PMC\\Bioinformatics\\"
file_names=os.listdir(link)
for i in file_names[:100]:
    f= open(link+i, encoding="utf8")
    text=f.read()
    s=re.search('==== Body(.*)==== Refs',text.replace("\n"," "))
    if not s is None:
        intro=intro+s.group(1)
    f.close()

In [70]:
np.unique([i for i in re.findall("([A-Z]+)",intro) if len(i)> 5])

array(['AAAAACCTTCATTTTATTTTATCTCCA', 'AAACACATCAGCAGCTTCAATACC',
       'AAAGGTTCAACT', 'AATAAA', 'AATAAACA', 'AATACGTAC', 'AAUAAA',
       'ACAACA', 'ACAACACTTAA', 'ACAACACTTAAAGGTTCAACTAGCAATA',
       'ACATTCCAGCCCCAAAACC', 'ACCOUNTS', 'ACKNOWLEDEGMENTS',
       'ACKNOWLEDGEMENT', 'ACKNOWLEDGEMENTS', 'ACKNOWLEDGMENTS',
       'ACTTTTGAGTTAGGCCCTTTTT', 'ADMINISTRATION', 'AGACGTG',
       'AGCAGCTCGTAATGCATCCT', 'AGGTAAG', 'AGGTAAGT', 'AGMIAL', 'AGTAGT',
       'AGTAGTG', 'AGTCAGTT', 'ALGORITHM', 'ALGORITHMIC', 'ALGORITHMS',
       'ALIGNING', 'ANALYSIS', 'APPENDIX', 'APPLICATION', 'APPLICATIONS',
       'APPROACH', 'APPROXMGD', 'AREVIR', 'ARGUMENTATION', 'ARGUMENTION',
       'ARGUMENTS', 'ASPECTS', 'ASTRAL', 'ATATAAA', 'ATATTAT', 'ATGTGCTA',
       'AUGIST', 'AVAILABILITY', 'BACKBONE', 'BACKGROUND', 'BACKRUB',
       'BASELINE', 'BATWING', 'BEHAVIOR', 'BENCHMARKING', 'BERNWARD',
       'BIGINT', 'BINARY', 'BIOACTIVITY', 'BIOLOGICAL', 'BIOLOGISTS',
       'BIOLOGY', 'BIOMEDICAL', 'B